In [2]:
import pandas as pd
import binascii

In [3]:
fname="/Users/jianlin/Documents/Project_STDF_wafermap/STDF_data/main_Lot_1_Wafer_1_Oct_13_09h33m41s_STDF.stdf"

In [197]:
with open(fname,mode='rb') as file:
    row_data = file.read()
str_row_data=str(row_data)
str_row_data

"b'\\x02\\x00\\x00\\n\\x02\\x04y\\x00\\x00\\x14\\xf8T\\xffWtdataformatter tester config; Ver: SmarTest s/w rev. 7.1.4.3 (E), 15-Oct-12; dataformatter 2.0.9 Oct 15 2012 18:37:15\\x82\\x00\\x01\\n\\xeaT\\xffW\\xf8T\\xffW\\x01 N \\xff\\xff \\x011\\x00\\x15localhost.localdomain\\t93000-SOC\\x05cp_32\\x00\\x00\\x07vincent\\x0593000 s/w rev. 7.1.4.3 (E), 15-Oct-12\\n\\x00\\x0525.25\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x14\\x00\\x01P\\x01\\x02\\x01\\x01\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x12\\x00\\x01<\\x01\\x00\\x01\\x00\\x03Xin\\x0510101\\x011\\x01\\x01\\x13\\x00\\x01<\\x02\\x00\\x02\\x00\\x04Xout\\x0510102\\x012\\x01\\x01\\x0e\\x00\\x01<\\x03\\x00\\x00\\x00\\x03VCC\\x0211\\x00\\x01\\x01\\x14\\x00\\x02\\x1e\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00 \\x00\\x80\\x00\\x80  \\x07\\x00\\x02\\n\\x01\\x02\\xf8T\\xffW\\x00\\x02\\x00\\x05\\n\\x01\\x01\\x12\\x00\\x01<\\x01\\x00\\

# Dict for convert

In [155]:
dict_code_ctype = {
    "U1": "B",
    "U2": "H",
    "U4": "I",
    "U8": "Q",
    "I1": "b",
    "I2": "h",
    "I4": "i",
    "I8": "q",
    "R4": "f",
    "R8": "d",
    "B1": "B",
    "C1": "c",
    "N1": "B"
}
dict_code_ctype['U1']

'B'

In [ ]:
dict_header_grep = {
    'FAR':'x02\x00\x00\n',
    'ATR':''
}

# Filter Method

In [231]:
# header of PIR
header_prr = struct.pack("BBBB", 26,0,5,20)
header_prr_2 = struct.pack("BBBB", 27,0,5,20)
header_pir = struct.pack("BBBB", 2,0,5,10)
str_prr = str(header_prr)
str_prr_2 = str(header_prr_2)
str_pir = str(header_pir)
str_prr

"b'\\x1a\\x00\\x05\\x14'"

In [232]:
key_prr=str_prr[2:len(str_prr)-1]
key_prr_2=str_prr_2[2:len(str_prr_2)-1]
key_pir=str_pir[2:len(str_pir)-1]

In [233]:
head_filter = str_row_data.replace(key_prr, "[PRR]").replace(key_prr_2, "[PRR]").replace(key_pir,"[PIR]") 
head_filter

"b'\\x02\\x00\\x00\\n\\x02\\x04y\\x00\\x00\\x14\\xf8T\\xffWtdataformatter tester config; Ver: SmarTest s/w rev. 7.1.4.3 (E), 15-Oct-12; dataformatter 2.0.9 Oct 15 2012 18:37:15\\x82\\x00\\x01\\n\\xeaT\\xffW\\xf8T\\xffW\\x01 N \\xff\\xff \\x011\\x00\\x15localhost.localdomain\\t93000-SOC\\x05cp_32\\x00\\x00\\x07vincent\\x0593000 s/w rev. 7.1.4.3 (E), 15-Oct-12\\n\\x00\\x0525.25\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x14\\x00\\x01P\\x01\\x02\\x01\\x01\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x12\\x00\\x01<\\x01\\x00\\x01\\x00\\x03Xin\\x0510101\\x011\\x01\\x01\\x13\\x00\\x01<\\x02\\x00\\x02\\x00\\x04Xout\\x0510102\\x012\\x01\\x01\\x0e\\x00\\x01<\\x03\\x00\\x00\\x00\\x03VCC\\x0211\\x00\\x01\\x01\\x14\\x00\\x02\\x1e\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00 \\x00\\x80\\x00\\x80  \\x07\\x00\\x02\\n\\x01\\x02\\xf8T\\xffW\\x00[PIR]\\x01\\x01\\x12\\x00\\x01<\\x01\\x00\\x01\\x00\\x03

In [236]:
PRR_select = head_filter.split("[PRR]")
print(len(PRR_select))
print(PRR_select[12])

14
\x01\x01\x00\x00\x00\x01\x00\x01\x00\xfa\xff\x00\x00\x01\x00\x00\x00\x0212\x052.802\x00[PIR]\x01\x01


corrspon, +1-1 not PIR

# Matching Method